<a href="https://colab.research.google.com/github/kkushal05/tutorials/blob/main/spark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
%%bash
sudo apt install -y mongodb > log
service mongodb start
mongo --eval 'db.runCommand({ connectionStatus: 1 })'

In [ ]:
%%bash

ps -ef | grep mongo

In [ ]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() 

In [16]:
!pip install pandas
!pip install pyspark
!pip install pymongo
!pip install sodapy
!pip install progressbar

In [17]:
import pandas as pd
from sodapy import Socrata
import pymongo
from pymongo import MongoClient
import pyspark
from pyspark.sql import SparkSession
import progressbar
from pyspark.sql import Row
import os

client = Socrata("data.cityofnewyork.us", None)
start = 0
chunk_size = 50
results = []
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

spark = SparkSession \
    .builder \
    .appName("mongodbtest1") \
    .master('local')\
    .config("spark.mongodb.output.uri", "mongodb:/127.0.0.1:27017/raw_data") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

mongo_client = MongoClient()
db = mongo_client['raw_data']

In [ ]:
from dateutil import parser

def load_data_for_nypd_call_for_service():
  start = 0
  chunk_size = 5000
  data_id = "n2zq-pubd";
  cols = "NYPD_PCT_CD, BORO_NM, TYP_DESC, ARRIVD_TS, ADD_TS"
  db.call_for_service.drop()
  record_count = client.get(data_id, where = "ARRIVD_TS between '2022-02-01T00:00:00' and '2022-03-01T00:00:00'" , select="COUNT(*)")
  print(record_count)
  print("Record count in  DB : " + str(db.call_for_service.count_documents({})))

  while True: 
    results = client.get(data_id, select=cols, where = "ARRIVD_TS between '2022-02-01T00:00:00' and '2022-03-01T00:00:00'", offset=start, limit=chunk_size)
    dataframe = spark.createDataFrame(results)
    dataframe = dataframe.na.drop()
    for item in dataframe.collect():
      item = item.asDict()
      add_ts = parser.parse(item['ADD_TS'])
      arrivd_ts = parser.parse(item['ARRIVD_TS'])
      item['response_time'] = (arrivd_ts - add_ts).total_seconds()
      db.call_for_service.insert_one(item)
    print("Record count in  DB : " + str(db.call_for_service.count_documents({})))
    start = start + chunk_size
    #if start > int(record_count[0]['COUNT']):
    if start > 10000:
        break

def store_precinct_average_for_nypd():
    records = db.call_for_service.aggregate([{'$group':{'_id':'$NYPD_PCT_CD','average_nypd':{'$avg':'$response_time'}}},{'$project':{'_id':0,'precinct':'$_id','average_nypd':{'$trunc':['$average_nypd']}}}])
    for item in records:
      db.precinct_averages.update_one({'precinct': item['precinct']},{'$set' : {'average_nypd' : item['average_nypd']}}, upsert=True)
    print(db.precinct_averages.count_documents({}))

load_data_for_nypd_call_for_service()
store_precinct_average_for_nypd()


In [28]:
 #----LOAD DATA FOR EMS BETWEEN A SPECIFIC DATE RANGE-------
import requests
import json
from pyspark.sql import functions

def load_ems_data():
  offset = 0
  limit = 5000
  data_id = "76xm-jjuj";
  cols = "INCIDENT_DATETIME, BOROUGH, POLICEPRECINCT, FIRST_ON_SCENE_DATETIME, ZIPCODE, incident_response_seconds_qy"
  where_clause = "$where=INCIDENT_DATETIME between '2022-01-01T00:00:00' and '2022-03-01T00:00:00'";
  response = requests.get("https://data.cityofnewyork.us/resource/76xm-jjuj.json?$select=COUNT(*)&"+where_clause)
  res_arr = json.loads(response.text)
  record_count = res_arr[0]['COUNT']
  db.ems.drop()
  print(record_count)
  while True:
      url = "https://data.cityofnewyork.us/resource/76xm-jjuj.json?" + where_clause + "&$limit="+str(limit)+"&$offset="+str(offset);
      results = requests.get(url)
      dataframe = spark.createDataFrame(json.loads(results.text))
      dataframe = dataframe.na.drop()
      for item in dataframe.collect():
        item = item.asDict()
        item['incident_response_seconds_qy'] = int(item['incident_response_seconds_qy'])
        db.ems.insert_one(item)
      offset = offset + limit
      if offset > 10000:
          break

def store_precinct_average_for_ems():
    records = db.ems.aggregate([{'$group':{'_id':'$policeprecinct','average_ems':{'$avg':'$incident_response_seconds_qy'}}},{'$project':{'_id':0,'precinct':'$_id','average_ems':{'$trunc':['$average_ems']}}}])
    for item in records:
      db.precinct_averages.update_one({'precinct': item['precinct']},{'$set' : {'average_ems' : item['average_ems']}}, upsert=True)
    print(db.precinct_averages.count_documents({}))

load_ems_data()
store_precinct_average_for_ems()

238503
78


In [62]:
 #----LOAD DATA FOR FIRE DEPT BETWEEN A SPECIFIC DATE RANGE-------
import requests

def load_fire_dept_data():
  offset = 0
  limit = 5000

  data_id = "8m42-w767";
  cols = "INCIDENT_DATETIME,INCIDENT_BOROUGH,POLICEPRECINCT,FIRST_ON_SCENE_DATETIME,INCIDENT_CLASSIFICATION,INCIDENT_RESPONSE_SECONDS_QY"
  record_count = client.get(data_id, where = "INCIDENT_DATETIME between '2021-02-01T00:00:00' and '2022-03-01T00:00:00'", select="COUNT(*)")
  print(record_count)
  db.fire_dept.drop()

  where_clause = "$where=INCIDENT_DATETIME between '2021-01-01T00:00:00' and '2022-03-01T00:00:00'";

  while True: 
      url = "https://data.cityofnewyork.us/resource/8m42-w767.csv?" + where_clause + "&$limit="+str(limit)+"&$offset="+str(offset)+"&$select="+cols;
      results = requests.get(url);
      csv_file = "/tmp/"+ data_id + ".csv"
      fh = open(csv_file, "wb")
      fh.write(results.content)
      fh.close()
      dataframe = spark.read.option("header", False).csv(csv_file)
      dataframe = dataframe.na.drop()
      header = dataframe.first()
      rows = dataframe.count()
      i=1
      for item in dataframe.collect():
        item = item.asDict()
        if item['_c0'] == "INCIDENT_DATETIME":
           continue
        item['_c5'] = int(item['_c5'])
        db.fire_dept.insert_one(item)
      offset = offset + limit
      if offset > 100:
          break

def store_precinct_average_for_fire():
    records = db.fire_dept.aggregate([{'$group':{'_id':'$_c2','average_fire':{'$avg':'$_c5'}}},{'$project':{'_id':0,'precinct':'$_id','average_fire':{'$trunc':['$average_fire']}}}])
    for item in records:
      db.precinct_averages.update_one({'precinct': item['precinct']},{'$set' : {'average_fire' : item['average_fire'] }}, upsert=True)
    print(db.precinct_averages.count_documents({}))

load_fire_dept_data()
store_precinct_average_for_fire()
docs = db.precinct_averages.find()

for doc in docs:
  print(doc)

[{'COUNT': '301031'}]
79
{'_id': ObjectId('62ed556e0e1badfa26615204'), 'precinct': None, 'average_nypd': 895.0, 'average_fire': None}
{'_id': ObjectId('62ed55fc0e1badfa26618cba'), 'precinct': '123', 'average_nypd': 296.0, 'average_ems': 470.0, 'average_fire': 338.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cbc'), 'precinct': '78', 'average_nypd': 417.0, 'average_ems': 374.0, 'average_fire': 248.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cbe'), 'precinct': '105', 'average_nypd': 1274.0, 'average_ems': 478.0, 'average_fire': 358.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cc0'), 'precinct': '22', 'average_nypd': 180.0, 'average_ems': 671.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cc2'), 'precinct': '114', 'average_nypd': 821.0, 'average_ems': 452.0, 'average_fire': 395.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cc4'), 'precinct': '49', 'average_nypd': 903.0, 'average_ems': 464.0, 'average_fire': 447.0}
{'_id': ObjectId('62ed55fc0e1badfa26618cc6'), 'precinct': '34', 'average_nypd': 1692.0, 'aver

In [ ]:
 #----LOAD RACIAL DATA -------
import requests

def load_race_data():
  offset = 0
  limit = 50
  data_id = "ik5a-xme5.csv"
  db.race_data.drop()

  url = "https://data.cityofnewyork.us/resource/" + data_id;
  results = requests.get(url);
  csv_file = "/tmp/"+ data_id + ".csv"
  fh = open(csv_file, "wb")
  fh.write(results.content)
  fh.close()
  dataframe = spark.read.csv(csv_file)
  dataframe = dataframe.na.drop()
  for item in dataframe.collect():
      print(item)
      db.race_data.insert_one(item.asDict())
  print("Record count in  DB : " + str(db.fire_dept.count_documents({})))

load_race_data()